Importing Libraries 

In [21]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 

from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import cross_validate, cross_val_predict, cross_val_score, LeaveOneOut, train_test_split
from sklearn.metrics import r2_score, confusion_matrix, mean_squared_error

Definitions 

In [80]:
# Calculation r2
def r2_from_sqerrs_and_values(squared_errors, y):
    return 1-squared_errors.mean()/(y.var())

# Calculation Mean Squared Error
def MSE_from_sqerrs_and_values(squared_errors, y):
    return -squared_errors.mean()

# Calculation Error
def error_from_sqerrs_and_values(squared_errors, y):
    return (np.exp(np.sqrt(-squared_errors.mean()))-1)*100

# Calculation Squared Error
def squared_errors(squared_errors, y):
    return np.sqrt(-1*squared_errors)


def metrics(predicted, labels):
    RMSE = mean_squared_error(labels,predicted, squared=False)
    R2 = r2_score(labels,predicted)
    var_pred = predicted.var()
    var_res = (labels-predicted).var()
    print("Root mean Squared Error is", RMSE)
    print("The R2 value is", R2)
    print("Variance of predictions is", var_pred)
    print("Variance of residuals is", var_res)

Importing Database and sorting by Type  

In [54]:
# Import the database csv file 
df = pd.read_csv('Kp Database.csv', encoding = 'ISO-8859-1')

# Do a sorting of the database and reset the index number
df = df.sort_values(by='Type') 

# Print the first few entries of the dataset 
print(df.head(5))

                    Monomer Abbreviation       Type  Acrylate  Test  \
0          n-Butyl acrylate            BA  Acrylate  Acrylate     1   
11         Stearyl acrylate            SA  Acrylate  Acrylate     0   
10  2-propylheptyl acrylate           PHA  Acrylate  Acrylate     0   
9           Methyl acrylate            MA  Acrylate  Acrylate     0   
8        iso-nonyl acrylate         INA-A  Acrylate  Acrylate     0   

                         Smiles           A     Ea   Kp25   Kp50  ...  \
0                 C=CC(OCCCC)=O  22100000.0  17.90  16154  28242  ...   
11  C=CC(OCCCCCCCCCCCCCCCCCC)=O  18600000.0  16.93  20107  34105  ...   
10    C=CC(OC[C@H](CCC)CCCCC)=O  10500000.0  16.41  14000  23364  ...   
9                    C=CC(OC)=O  14100000.0  17.30  13129  22527  ...   
8          C=CC(OCCCCCCC(C)C)=O  13500000.0  16.54  17080  28620  ...   

    Spider_RI_P  Spider_D  Spider_D_P  Spider_ACDlogP  Spider_P  Spider_ST  \
0         1.418     0.895         0.9            2.39   

Selecting data

In [75]:
# Create a dataframe and select the parameters/the collumns you want to use 

data = df[['Weight_equal','Spider_BP_P','Spider_RI_P','Spider_D_P','Spider_ACDlogP','Spider_P','Spider_ST','Spider_VP', 'Monomer','A','Acrylate','Ea', 'Type','R', 'Mr', 'DP', 'BPK', 'MPK', 'GFE','Kp25','Kp50','Kp75','Kp100', 'ln(Kp25)', 'ln(Kp50)', 'ln(Kp75)', 'ln(Kp100)', 'a_value', 'Hbond', 'Hacc', 'Hdon', 'A1*', 'A2*']]

# This selection has more parameters, but also more empty cells (n = 13)
#data = df[['Weight_equal','Spider_BP','Spider_BP_P','Spider_RI','Spider_RI_P','Spider_D','Spider_D_P','Spider_ACDlogP','Spider_P','Spider_ST','Spider_VP', 'Monomer','A','Acrylate','Ea', 'Type','R', 'Mr', 'DP', 'BPK', 'MPK', 'GFE','Kp25','Kp50','Kp75','Kp100', 'ln(Kp25)', 'ln(Kp50)', 'ln(Kp75)', 'ln(Kp100)', 'a_value', 'Hbond', 'Hacc', 'Hdon', 'A1*', 'A2*']]


# Further selection of the data by type 
#data = data[(data.Type == 'Acrylate') | (data.Type == 'Methacrylate')| (data.Type == 'H-Binding monomers')]
#data = data[(data.Type == 'Acrylate') | (data.Type == 'Methacrylate')| (data.Type == 'Other')]
#data = data[(data.Type == 'Acrylate') | (data.Type == 'Methacrylate')|]

# Leave out all the rows that do not have an entry 
data = data.dropna(how = 'any',axis = 0).reset_index(drop = True)

# Reset the index
data = data.reset_index()

print(data)


    index  Weight_equal  Spider_BP_P  Spider_RI_P  Spider_D_P  Spider_ACDlogP  \
0       0             1        145.0        1.418         0.9            2.39   
1       1             1        400.2        1.452         0.9            9.83   
2       2             1        266.1        1.439         0.9            5.39   
3       3             1         80.2        1.390         0.9            0.79   
4       4             1        247.3        1.437         0.9            4.86   
5       5             1        244.5        1.491         1.0            4.22   
6       6             1        133.0        1.418         0.9            2.02   
7       7             1        174.6        1.420         1.0            1.02   
8       8             1        442.3        1.455         0.9           11.42   
9       9             1        385.6        1.451         0.9            9.30   
10     10             1        228.7        1.517         1.1            2.27   
11     11             1     

Selecting X and Y 

In [76]:
# Select the parameters for X
X_train = data[['Mr','a_value', 'R', 'Hacc', 'Hdon','A1*', 'A2*', 'Spider_P']]
#X_train = data[['Mr','a_value']]
#X_train = data[['Mr', 'DP', 'BPK', 'MPK', 'GFE']]

#Select the parameters for y
y_train = data['ln(Kp25)']
y_train25 = data['ln(Kp25)']
y_train50 = data['ln(Kp50)']
y_train75 = data['ln(Kp75)']
y_train100 = data['ln(Kp100)']

Cross Validation using a RIDGE regression 

In [78]:
res = cross_validate(RidgeCV(alphas=(10**np.linspace(10,-2,300)*0.5)), X_train, y_train, cv=LeaveOneOut(), scoring='neg_mean_squared_error', return_train_score=True, return_estimator=True, fit_params={'sample_weight': data.Weight_equal})

sum = np.zeros(len(res["estimator"]))

for est in res["estimator"]:
    sum = sum + est.predict(X_train)

ans = sum / len(res["estimator"])
print(ans)


[ 9.45888119  9.84275516  9.65915628  9.38164887  9.60688182  9.75786617
  9.63006759  9.87348836  9.92510232  9.82708587  9.67713055  9.96012061
  9.60517956  9.59375962 10.62137307  6.50309511 10.19313821 10.43028308
  6.44593811  6.32492468  6.40127972  6.04657749  5.94885373  6.19969296
  6.05501896  5.96965043  5.98381163  6.16267972  6.24649498  6.17862526
  6.24436468  6.51864517  6.50400512  4.59929358  8.59013628]


Cross Validation using a LINEAR regression 

In [79]:
res = cross_validate(LinearRegression(), X_train, y_train, cv=LeaveOneOut(), scoring='neg_mean_squared_error', return_train_score=True, return_estimator=True, fit_params={'sample_weight': data.Weight_equal})

sum = np.zeros(len(res["estimator"]))

for est in res["estimator"]:
    sum = sum + est.predict(X_train)

ans = sum / len(res["estimator"])
print(ans)

[ 9.45042067  9.84340776  9.68760471  9.37766379  9.59934488  9.7159305
  9.79297807  9.85637056  9.92592977  9.82769264  9.6939735   9.9638689
  9.62877883  9.4569108  10.68427647  6.59276943 10.28577703 10.36754296
  6.51123409  6.27382994  6.3988447   6.03374254  5.93562104  6.19243188
  6.06627495  5.89544581  5.98993417  6.18381108  6.23680663  6.18018739
  6.25194232  6.51930584  6.47288265  3.94396628  8.49742636]


Calculating the metrics by comparing the predicted values to their original values 

In [82]:
metrics(ans, y_train)

Root mean Squared Error is 0.23341401903862496
The R2 value is 0.9846567133100715
Variance of predictions is 3.6093573250256115
Variance of residuals is 0.05591739110840906


Calculate predictions for different temperatures

In [86]:
# Result for 25 degrees
res25 = cross_validate(RidgeCV(alphas=(10**np.linspace(10,-2,300)*0.5)), X_train, y_train25, cv=LeaveOneOut(), scoring='neg_mean_squared_error', return_train_score=True, return_estimator=True, fit_params={'sample_weight': data.Weight_equal})

sum = np.zeros(len(res25["estimator"]))

for est in res25["estimator"]:
    sum = sum + est.predict(X_train)

rid_predsR25 = sum / len(res25["estimator"])
print(rid_predsR25)


# Result for 50 degrees
res50 = cross_validate(RidgeCV(alphas=(10**np.linspace(10,-2,300)*0.5)), X_train, y_train50, cv=LeaveOneOut(), scoring='neg_mean_squared_error', return_train_score=True, return_estimator=True, fit_params={'sample_weight': data.Weight_equal})

sum = np.zeros(len(res50["estimator"]))

for est in res50["estimator"]:
    sum = sum + est.predict(X_train)

rid_predsR50 = sum / len(res50["estimator"])
print(rid_predsR50)


# Result for 75 degrees
res75 = cross_validate(RidgeCV(alphas=(10**np.linspace(10,-2,300)*0.5)), X_train, y_train75, cv=LeaveOneOut(), scoring='neg_mean_squared_error', return_train_score=True, return_estimator=True, fit_params={'sample_weight': data.Weight_equal})

sum = np.zeros(len(res75["estimator"]))

for est in res75["estimator"]:
    sum = sum + est.predict(X_train)

rid_predsR75 = sum / len(res75["estimator"])
print(rid_predsR75)


# Result for 100 degrees

res100 = cross_validate(RidgeCV(alphas=(10**np.linspace(10,-2,300)*0.5)), X_train, y_train100, cv=LeaveOneOut(), scoring='neg_mean_squared_error', return_train_score=True, return_estimator=True, fit_params={'sample_weight': data.Weight_equal})

sum = np.zeros(len(res100["estimator"]))

for est in res100["estimator"]:
    sum = sum + est.predict(X_train)

rid_predsR100 = sum / len(res100["estimator"])
print(rid_predsR100)

[ 9.45888119  9.84275516  9.65915628  9.38164887  9.60688182  9.75786617
  9.63006759  9.87348836  9.92510232  9.82708587  9.67713055  9.96012061
  9.60517956  9.59375962 10.62137307  6.50309511 10.19313821 10.43028308
  6.44593811  6.32492468  6.40127972  6.04657749  5.94885373  6.19969296
  6.05501896  5.96965043  5.98381163  6.16267972  6.24649498  6.17862526
  6.24436468  6.51864517  6.50400512  4.59929358  8.59013628]
[ 9.97728308 10.30009457 10.15730816  9.91410648 10.10435746 10.26243342
 10.17784635 10.36491039 10.36921878 10.28996329 10.17370495 10.40079525
 10.11188178 10.04625593 11.04107782  7.17546812 10.62460975 10.85526625
  7.11802124  7.02360247  7.0523095   6.75438205  6.6754245   6.88740403
  6.77249912  6.6724807   6.70815445  6.86305394  6.92441549  6.89427577
  6.93505277  7.15301017  7.20344882  5.59944573  9.07832822]
[10.42112085 10.69185404 10.5845684  10.37006396 10.53031976 10.69397486
 10.65031547 10.78483189 10.74966647 10.68647583 10.5989289  10.77836328


Make arrhenius plot and calculate the slope and activation energy 

In [87]:
R_slope_list = []
R_intercept_list = []

rid_predsR25_list = rid_predsR25.tolist()
rid_predsR50_list = rid_predsR50.tolist()
rid_predsR75_list = rid_predsR75.tolist()
rid_predsR100_list = rid_predsR100.tolist()

for (i, j, k, l) in zip(rid_predsR25_list, rid_predsR50_list, rid_predsR75_list, rid_predsR100_list):
    y = [i, j, k, l]
    x = [1/(25+273.15), 1/(50+273.15), 1/(75+273.15), 1/(100+273.15)]
    slope, intercept = np.polyfit(x,y,1)
    R_slope_list.append(slope)
    R_intercept_list.append(intercept)
    
df_R_s = pd.DataFrame({'Slope Predicted': R_slope_list})
df_R_s['Ea Predicted'] = -df_R_s['Slope Predicted']*8.314/1000
df_R_s['Ea Experimental'] = data[['Ea']]
df_R_s['Type'] = data[['Type']]
df_R_s['Name'] = data[['Monomer']]
df_R_s['Acrylate'] = data[['Acrylate']]

df_R_i = pd.DataFrame({'Intercept Predicted': R_intercept_list})
df_R_i['A Predicted'] = np.exp(df_R_i['Intercept Predicted'])
df_R_i['A Experimental'] = data[['A']]
df_R_i['ln(A Predicted)'] = np.log(df_R_i['A Predicted'])
df_R_i['ln(A Experimental)'] = np.log(df_R_i['A Experimental'])
df_R_i['Type'] = data[['Type']]
df_R_i['Acrylate'] = data[['Acrylate']]

#print(df_R_s)
print(df_R_i)

    Intercept Predicted   A Predicted  A Experimental  ln(A Predicted)  \
0             16.162671  1.045584e+07    2.210000e+07        16.162671   
1             15.751137  6.928383e+06    1.860000e+07        15.751137   
2             16.102072  9.841037e+06    1.050000e+07        16.102072   
3             16.269115  1.163019e+07    1.410000e+07        16.269115   
4             16.037744  9.227917e+06    1.350000e+07        16.037744   
5             16.276342  1.171455e+07    4.810000e+06        16.276342   
6             16.731752  1.847173e+07    2.210000e+07        16.731752   
7             16.215955  1.102808e+07    6.300000e+06        16.215955   
8             15.661064  6.331603e+06    3.220000e+06        15.661064   
9             15.807549  7.330462e+06    8.150000e+06        15.807549   
10            16.094399  9.765822e+06    1.280000e+07        16.094399   
11            15.651397  6.270688e+06    5.350000e+06        15.651397   
12            16.159113  1.041870e+07 

In [ ]:
metrics(df_R_s['Ea Predicted'], df_R_s['Ea Experimental'])
metrics(df_R_i['ln(A Predicted)'], df_R_i['ln(A Experimental)'])

Root mean Squared Error is 0.455307776673516
The R2 value is 0.987345438377479
Variance of predictions is 16.559358509228847
Variance of residuals is 0.22124148999506277
Root mean Squared Error is 0.17403061963345237
The R2 value is 0.9669053033632509
Variance of predictions is 0.9323030068963246
Variance of residuals is 0.03275077826723921


In [89]:
df_test = pd.read_csv('Test Database5.csv', encoding = 'ISO-8859-1')
print(df_test)

   ï»¿Test             Smiles      Mr        DP      BPK      BPC     MPK  \
0        0       CCCOC(=O)C=C  114.14  2.231852  397.247  124.097  197.45   
1        0        CCOC(=O)C=C  100.12  2.300045  373.693  100.543  186.18   
2        0  C=CC(=O)OC1CCCCC1  154.21  3.851127  471.471  198.321  238.64   
3        0    CCCOC(=O)C(=C)C  128.17  1.994317  435.798  162.648  206.03   

      MP      GFE   a_value  R  Hbond  Hacc  Hdon  A1*  A2*  Spider_P  
0 -75.70   82.709  0.000000  0      0     0     0 -2.8    1      12.4  
1 -86.97   64.877  0.000000  0      0     0     0 -2.8    1      10.6  
2 -34.51  125.539  0.000000  0      0     0     0 -2.8    1      17.2  
3 -67.12  100.506 -1.553092  0      0     0     0 -2.8  647      14.2  


In [90]:
df = pd.read_csv('Kp Database.csv', encoding = 'ISO-8859-1')
df = df.sort_values(by='Type') #sort by type to make copypaste easier
df = df.reset_index()

#data = df[['Type','Mr', 'DP', 'BPK', 'GFE', 'R', 'Hacc', 'Hdon', 'A1*', 'A2*', 'Spider_P', 'ln(Kp25)', 'a_value', 'Spider_BP','Spider_RI','Spider_D']]
data = df[['Type', 'Mr', 'DP', 'BPK', 'GFE', 'R', 'Hacc', 'Hdon', 'A1*', 'A2*', 'Spider_P', 'ln(Kp25)', 'a_value']]
#data = data[(data.Type == 'Acrylate') | (data.Type == 'Methacrylate')]
data = data.dropna(how = 'any',axis = 0).reset_index(drop = True)
data = data.reset_index()
print(data)

X_train = data[['Mr', 'DP', 'BPK', 'GFE', 'a_value','R', 'Hacc', 'Hdon', 'A1*', 'A2*', 'Spider_P']]
y_train = data['ln(Kp25)']
X_test = df_test[['Mr', 'DP', 'BPK', 'GFE', 'a_value', 'R', 'Hacc', 'Hdon', 'A1*', 'A2*', 'Spider_P']]


#X_train = data[['Mr','a_value', 'R', 'Hacc', 'Hdon','A1*', 'A2*', 'Spider_P']]
#y_train = data['ln(Kp25)']
#X_test = df_test[['Mr','a_value', 'R', 'Hacc', 'Hdon','A1*', 'A2*', 'Spider_P']]



print(X_train)

    index                Type       Mr        DP      BPK     GFE          R  \
0       0            Acrylate  128.170  2.448535  419.897 -275.89  12.749918   
1       1            Acrylate  324.500  2.670718  644.497 -158.01  12.708460   
2       2            Acrylate  212.330  2.498485  526.538 -227.81  12.438983   
3       3            Acrylate   86.090  2.230716  349.235 -301.15  12.708460   
4       4            Acrylate  198.300  2.499832  508.767 -236.23  12.708460   
5       5            Acrylate  208.300  2.369456  515.978 -142.37  12.418254   
6       6            Acrylate  128.170  2.271456  397.266 -273.05  11.278159   
7       7            Acrylate  144.170  3.909485  442.469 -380.89  13.433975   
8       8            Acrylate  366.600  2.762160  679.309 -132.75  12.708460   
9       9            Acrylate  310.500  3.173710  632.893 -166.43  12.667002   
10     10            Acrylate  162.180  2.401429  506.117 -138.22  13.081582   
11     11            Acrylate  380.600  

In [95]:
predict_ridge = RidgeCV(alphas=(10**np.linspace(10,-2,300)*0.5)).fit(X_train, y_train)
best_alpha = predict_ridge.alpha_
print(best_alpha)

0.026386271762417663


In [97]:
model = Ridge(alpha=best_alpha)
model.fit(X_train, y_train)
yhat = model.predict(X_test)
print(yhat)

ans = np.exp(yhat)
print(ans)
print(best_alpha)


[10.52473873 10.46141291 10.82456464  7.19186629]
[37225.10680117 34940.88477799 50239.89140231  1328.58041023]
0.026386271762417663
